In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import welch
import joblib
import os
from sklearn.preprocessing import StandardScaler
import ipywidgets as widgets
from IPython.display import display
from scipy.stats import skew, kurtosis


In [5]:
from google.colab import files
uploaded = files.upload()

# Load the first file
filename = next(iter(uploaded))
df = pd.read_csv(filename)
print(df.head())

# Calculate sampling rate
time_diffs = np.diff(df['Time (s)'])
sampling_rate = int(round(1 / np.median(time_diffs)))
samples_per_window = sampling_rate
num_windows = len(df) // samples_per_window

segments = []
for i in range(num_windows):
    start = i * samples_per_window
    end = start + samples_per_window
    segment = {
        'start_time': df['Time (s)'].iloc[start],
        'X': df['X-Axis (m/s2)'].iloc[start:end].values,
        'Y': df['Y- Axis (m/s2)'].iloc[start:end].values,
        'Z': df['Z-Axis (m/s2)'].iloc[start:end].values
    }
    segments.append(segment)

def extract_features(segment, fs):
    features = {}
    for axis in ['X', 'Y', 'Z']:
        sig = segment[axis]
        features['rms'] = np.sqrt(np.mean(sig**2))
        features['std'] = np.std(sig)
        features['peak_to_peak'] = np.ptp(sig)
        features['time_skew'] = skew(sig)
        features['time_kurtosis'] = pd.Series(sig).kurtosis()




        freqs, psd = welch(sig, fs=fs)
        band_mask = (freqs >= 50) & (freqs <= 200)
        band_energy = np.sum(psd[band_mask])
        total_energy = np.sum(psd)

        features['band_energy_ratio'] = band_energy / total_energy if total_energy > 0 else 0
        features['dominant_freq'] = freqs[np.argmax(psd)]
        features['spectral_centroid'] = np.sum(freqs * psd) / np.sum(psd)
        features['peak_count'] = np.sum(psd > np.mean(psd))
        features['spectral_kurtosis'] = kurtosis(psd)

    return features

features_list = [extract_features(seg, sampling_rate) for seg in segments]
features_df = pd.DataFrame(features_list)

# Upload and load model
from google.colab import files
uploaded_model = files.upload()
model = joblib.load(next(iter(uploaded_model)))

# Predict without scaling
features_df['prediction'] = model.predict(features_df)

def interactive_plot(index):
    seg = segments[index]
    label = features_df.iloc[index]['prediction']

    fig, axs = plt.subplots(3, 2, figsize=(12, 8))
    axes = ['X', 'Y', 'Z']
    for i, axis in enumerate(axes):
        sig = seg[axis]
        time = np.linspace(0, 1, len(sig))
        freqs, psd = welch(sig, fs=sampling_rate)

        axs[i, 0].plot(time, sig)
        axs[i, 0].set_title(f"{axis}-axis Time Signal")

        axs[i, 1].semilogy(freqs, psd)
        axs[i, 1].set_title(f"{axis}-axis FFT Spectrum")

    plt.suptitle(f"Segment {index} — {'Chatter' if label else 'Stable'}", fontsize=14)
    plt.tight_layout()
    plt.show()

segment_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(segments) - 1,
    step=1,
    description='Segment:',
    continuous_update=False
)

widgets.interact(interactive_plot, index=segment_slider)



Saving a1_cleaned.csv to a1_cleaned.csv
   Time (s)  X-Axis (m/s2)  Y- Axis (m/s2)  Z-Axis (m/s2)
0  0.000000       0.016907        0.047448      -0.017904
1  0.002470       0.009565        0.001118       0.092957
2  0.004943      -0.111627       -0.048665      -0.116914
3  0.007416       0.117938        0.051428      -0.090291
4  0.009891       0.048750        0.085182       0.029407


Saving voting_model.pkl to voting_model.pkl


interactive(children=(IntSlider(value=0, continuous_update=False, description='Segment:', max=13), Output()), …

<function __main__.interactive_plot(index)>